In [2]:
import time
import random
import requests
import selenium
from string import ascii_lowercase
from selenium.webdriver import Firefox, Chrome
from bs4 import BeautifulSoup
import pandas as pd
import string
import numpy as np
from selenium.webdriver.common.keys import Keys
import pymongo

In [8]:
browser = Chrome()

In [10]:
def search_allrecipes(search_item, browser, typo_rate=0.1):
    """Goes to the Allrecipes.com homepage and searches for item specified by 'search_item'"""
    url='https://www.allrecipes.com'
    browser.get(url)
    sel = 'input#searchText'
    search_bar = browser.find_element_by_css_selector(sel)
    search_bar.click()
    type_item(search_item, search_bar, typo_rate)
    search_bar.send_keys('\n')

In [11]:
def type_item(item, field, typo_rate=0.1):
    for char in item:
        if np.random.random() < typo_rate:
            field.send_keys(make_typo(char))
            time.sleep(0.2 * np.random.random() + typo_rate)
            field.send_keys('\b')
        field.send_keys(char)
        time.sleep(0.05 * np.random.random() + typo_rate)

In [12]:
def make_typo(key, repeat=True):
    '''Returns a letter adjacent to (or identical to, if repeat=True) the input key.'''
    key = key.lower()
    row2 = list('qwertyuiop[')
    row3 = list("asdfghjkl;")
    row4 = list('zxcvbnm,.')
    adj_keys = [key] if repeat else []
    if key in row2:
        idx = row2.index(key)
        if idx > 0:
            adj_keys += [row2[idx-1] , row2[idx+1]] + row3[idx-1:idx+1]
        else:
            adj_keys += list('was')
    elif key in row3:
        idx = row3.index(key)
        if idx > 0:
            adj_keys += row2[idx:idx+2] + [row3[idx-1] , row3[idx+1]] + row4[idx-1:idx+1]
        else:
            adj_keys += list('qwsz')
    elif key in row4:
        idx = row4.index(key)
        if idx > 0:
            adj_keys += row3[idx:idx+2] + [row4[idx-1] , row4[idx+1]]
        else:
            adj_keys += list('asx')
    return np.random.choice(adj_keys)

In [13]:
def clear_field(field):
    contents = field.get_attribute('value')
    for _ in contents:
        field.send_keys('\b')
        time.sleep(0.02 * np.random.random() + 0.05)

In [14]:
search_allrecipes('chili', browser)

In [72]:
browser.execute_script('window.scrollTo(0, document.body.scrollHeight - 1000);')

In [15]:
def get_search_results(browser):
    '''
    Finds and returns all recipe ids, names, and hyperlinks on current browser page.
    '''
    sel = 'article.fixed-recipe-card'
    search_results = browser.find_elements_by_css_selector(sel)
    hrefs = []
    names = []
    for element in search_results:
        try:
            sel = 'div.fixed-recipe-card__info h3 a'
            info = element.find_element_by_css_selector(sel)
            href = info.get_attribute('href')
            hrefs.append(href.split('/?')[0])
            names.append(info.text)
        except:
            continue
    ids = []
    for href in hrefs:
        id_and_name = href.split('recipe/')[1]
        ids.append(int(id_and_name.split('/')[0]))
    return ids, names, hrefs

In [16]:
ids, names, hrefs = get_search_results(browser)

In [85]:
def populate_search_page(browser, num_pages=10, scroll_delay=4):
    '''Populates browser page with specified number of pages of search results'''
    for _ in range(num_pages):
        try:
            sel = 'button#btnMoreResults'
            more_button = browser.find_element_by_css_selector(sel)
            more_button.click()
        except:
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight - 1000);')
        time.sleep(scroll_delay + scroll_delay*np.random.random())

In [81]:
populate_search_page(browser, num_pages=5)

In [3]:
mc = pymongo.MongoClient()

In [4]:
test_db = mc['recipes_test']

In [19]:
results_coll = test_db['search_results']

In [20]:
def add_results_to_collection(browser, collection):
    ids, names, hrefs = get_search_results(browser)
    for i, name, href in zip(ids, names, hrefs):
        # If 'id' is not in database, add entry
        if len(list(collection.find({'id':i}))) == 0:
            item = {'id':i,
                    'name': name,
                    'href': href,
                    'viewed': 0}
            collection.insert_one(item)

In [23]:
add_results_to_collection(browser, results_coll)

In [25]:
def mark_as_viewed(recipe_id, collection):
    collection.update_one({'id':recipe_id},
                          {"$set":{'viewed': 1}}, upsert=False)

In [26]:
mark_as_viewed(78299, results_coll)

In [29]:
len(list(results_coll.find()))

20